# Group 07 - Code - Rotten Tomatoes Web Scraping
## Vedenikova_Vitalia_StudentB

In [ ]:
#Importing the necessary packages
import pandas as pd
import numpy as np
import pickle
import sys
import re
import bs4
import requests
from bs4 import BeautifulSoup
import time
import json
import os
import csv
import warnings
from scipy import stats
import matplotlib.pyplot as plt
from requests_html import HTMLSession
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from ast import literal_eval
pd.set_option('display.max_colwidth', None) #allows to see all the columns when working with data frames.

## 2. URLs to access

### 2.1 Generating the URLs

In [ ]:
#Import Student A's data frame
dfA= pd.read_csv("../Code/Data/StudentA_Early_Stage_1.csv")
dfA = dfA.rename(columns={'Movie Title':'Movie_Title'})

In [ ]:
#Import Student C's data frame and merge it with Student A's 
dfC= pd.read_csv("../Code/Data/StudentC_Early_Stage_1.csv")
dfC = dfC.rename(columns={'Title':'Movie_Title', 'Year':'Release_Year'})
df = pd.merge(dfA, dfC, on='Movie_Title', how='right')

#Keep only the useful columns
df=df.drop(df.loc[:, 'Released':'BoxOffice'].columns, axis=1)
df=df.drop(['Unnamed: 0_x','Oscar Ceremony Year','Unnamed: 0_y'], axis=1)
df

In [ ]:
#Define the URLs that we will be accessing

def generate_movie_links(df) -> None:
    """function to add columns to the dataframe, reformat the movie names and create an URL for each movie in the dataframe"""
    df.insert(2,"URL_Movie_Names", True)
    df.insert(3,"Rottentomatoes_URLs", True)
    df['URL_Movie_Names'] = df['Movie_Title'].str.lower() 
    df['URL_Movie_Names'] = df['URL_Movie_Names'].str.split('(').str[0] 
    df['URL_Movie_Names'] = df['URL_Movie_Names'].str.replace(' ', '_')
    df['URL_Movie_Names'] = df['URL_Movie_Names'].str.rstrip('_') 


    Char_replacements = [('&', 'and'),('é', 'e'),('á', 'a'),('à', 'a'),('í', 'i'),('ì', 'i'),('ǎ', 'a'),('ó', 'o'),('...', '_'),
                          ('_-_', '_')]

    for i,j in Char_replacements:
        df['URL_Movie_Names'] = df['URL_Movie_Names'].str.replace(i, j)

    
    URL_replacements = [('13_hours', '13_hours_the_secret_soldiers_of_benghazi'),('birdman_or', 'birdman'),('spider-man', 'spider_man'),
                        ('spider-verse', 'spider_verse'),('maggie_simpson_in', 'the_simpsons'),('ennemis_interieurs', 'enemies_within'),
                        ('everything_will_be_okay', 'grill_gut_alles_gut'),('the_white_tiger', 'the_white_tiger_2020'),
                        ('half-blood', 'half_blood'),('the_wife', 'the_wife_2018'),('lifeboat', 'lifeboat_2019'),
                        ('kung_fu_panda_2', 'kung_fu_panda_the_kaboom_of_doom'),('les_mis_rables', 'les_miserables'),
                        ('star_wars_episode_viii_the_last_jedi', 'star_wars_the_last_jedi'),('star_wars_episode_ix_the_rise_of_skywalker', 'star_wars_the_rise_of_skywalker'),
                        ('music_by_prudence', 'music_by_prudence_2011'),('once_upon_a_time__in', 'once_upon_a_time_in'),
                        ('the_baader_meinhof_complex', 'baader_meinhof_complex'),('the_curious_case_of_benjamin_button', 'curious_case_of_benjamin_button'),
                        ('the_last_station', 'last_station'),('golda', 'golda_2022'),('the_milk_of_sorrow', 'milk_of_sorrow'),('the_young_victoria', 'young_victoria'),
                        ('undefeated', 'undefeated_2012'),('one_night_in_miami_', 'one_night_in_miami'),('ala_kachuu', 'ala_kachuu_take_and_run'),('ave_maria', '771428498'),
                        ('changeling', '1191742_changeling'),('glass_onion', 'glass_onion_a_knives_out_mystery'),('joanna', '771467811'),('hellboy_ii', 'hellboy_2'),
                        ('the_100_yearold', 'the_100_year_old'),('the_betrayal', 'nerakhoon'),('the_present', 'the_present_0000'),('the_reader', 'reader'),
                        ('the_secret_in_their_eyes', 'secret_in_their_eyes'),('tick_tick__boom', 'tick_tick_boom')]
                        
    for i,j in URL_replacements:
        df['URL_Movie_Names'] = df['URL_Movie_Names'].str.replace(i, j)



    lsch = "!?.’:',+/-\"" 
    for ch in lsch:
        df['URL_Movie_Names'] = df['URL_Movie_Names'].str.replace(ch, '')
    
  
    Additional_replacements = [('chinas_unnatural_disaster_the_tears_of_sichuan_province', 'chinas-unnatural-disaster-the-tears-of-sichuan-province'),
                               ('encounters_at_the_end_of_the_world', '1194818-encounters_at_the_end_of_the_world'),('fantastic_mr_fox', '1197696-fantastic_mr_fox'),
                               ('how_they_donut_a_matter_of_loaf_and_death', 'wallace_and_gromit_a_matter_of_loaf_and_death_pands'),
                               ('lavatory_lovestory', 'ubornaya-istoriya-lyubovnaya-istoriya'),('pitääkö_mun_kaikki_hoitaa', 'pitaako_mun_kaikki_hoitaa'),
                               ('smile_pinki', 'smile-pinki'),('the_100yearold_man', 'the_100_year_old_man'),('the_conscience_of_nhem_en', 'the-conscience-of-nhem-en'),
                               ('the_duchess', '10009493-duchess'),('the_final_inch', 'the-final-inch'),('the_last_truck_closing_of_a_gm_plant', 'the-last-truck-the-closing-of-a-gm-plant'),
                               ('the_lovely_bones', '1189344-lovely_bones'),('the_wolfman', '1194949-wolfman'),('tron_legacy', '10011582-tron_legacy'),
                               ('winters_bone', '10012136-winters_bone'),('wreckit_ralph', 'wreck_it_ralph'),('xmen_days_of_future_past', 'x_men_days_of_future_past'),
                               ('yespeople', 'ja_folkid'),('dimanchesunday', 'dimanche'),('ninetyfive_senses', 'ninety_five_senses'),
                               ('the_most_dangerous_man_in_america_daniel_ellsberg_and_the_pentagon_papers', 'daniel_ellsberg_doc'),
                               ('precious_based_on_the_novel_push_by_sapphire', 'precious'),('shangchi', 'shang_chi'),
                               ('star_wars_the_force_awakens', 'star_wars_episode_vii_the_force_awakens'),('the_cove', '1208882-cove'),
                               ('wall·e', 'wall_e'),
                               ('borat_subsequent_moviefilm_delivery_of_prodigious_bribe_to_american_regime_for_make_benefit_once_glorious_nation_of_kazakhstan', 'borat_subsequent_moviefilm'),
                               ('alice_in_wonderland', '1221547-alice_in_wonderland'),('a_single_man', '1218217-single_man'),
                               ('burma_vj_reporting_from_a_closed_country', 'burma-vj'),('departures', '10010675-departures'),
                               ('kontiki', 'kon_tiki'),('the_house_of_small_cubes', 'la-maison-en-petits-cubes'),
                               ('the_visitor', '10008820-visitor'),('wanted', '1174279-wanted'),
                               ('watu_wote_all_of_us', 'watu_wote'),('happygolucky', '1195103-happy_go_lucky'),
                               ('salt', '1213717-salt')]
    
    for i,j in Additional_replacements:
        df['URL_Movie_Names'] = df['URL_Movie_Names'].str.replace(i, j)
    
    #create the URLs
    df['Rottentomatoes_URLs'] = [ f'https://www.rottentomatoes.com/m/{x}' for x in df['URL_Movie_Names']] #create the URLs
    
generate_movie_links(df)

Here below we see the sort of result we can expect from running the above code:

In [ ]:
df[df['Movie_Title'] == 'Changeling']

### 2.2 Fixing the URLs

**IMPORTANT NOTE** : As of 18.04.24 the below will no longer work with the URLs we previously defined since Rotten Tomatoes updated the movies' pages. 
As a consequence, we will be using a handful of links gathered from https://web.archive.org/ to demonstrate how the code runs.

In [ ]:
df= pd.read_csv("../Code/Data/df_waybackmachine_links.csv") # <---- this is the csv we will be using to show the code works

In [ ]:
#Trying to fix the URLs by using a separate function

myheaders = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) \ '
              'AppleWebKit/537.36 (KHTML, like Gecko) \ ' 		        
              'Chrome/84.0.4147.89 Safari/537.36'}

headers = {'User-Agent': myheaders}


def fix_urls(df):
    """function that fixes defective URLs in the dataframe by adding the release year at the end of the link and separating the links
    still defective for further investigation. There are two situations in which we add the year: 1) the link itself is defective since
    we retrieve nothing for movie data or 2)the release year on the movie page does not match the official release year of the film"""
    
    for row_index, row in df.iterrows(): 
        link = row['Rottentomatoes_URLs']
        page_text = requests.get(link, headers=myheaders).text
        soup = BeautifulSoup(page_text,'lxml')
        movie_characteristics = soup.find('div', class_= "panel-body content_body") 
        movie_release_year= soup.find('p', class_= "info", attrs={'data-qa':'score-panel-subtitle'})
        
        try:
            if(movie_characteristics == None):
                new_link = link+'_'+ str(row['Release_Year']) 
                df.loc[row_index,'Rottentomatoes_URLs'] =  new_link 
                page_text = requests.get(new_link, headers=myheaders).text
                soup = BeautifulSoup(page_text,'lxml')
                movie_characteristics = soup.find('div', class_= "panel-body content_body")
                if (movie_characteristics == None): 
                    with open('links_to_fix.csv', 'a', newline='') as file:
                        file.write(f"{df.loc[row_index,'Movie_Title']}, {new_link},\n")
                    continue
            elif(int(movie_release_year.text[0:4]) != int(row['Release_Year'])):
                new_link = link+'_'+ str(row['Release_Year']) 
                df.loc[row_index,'Rottentomatoes_URLs'] =  new_link 
                page_text = requests.get(new_link, headers=myheaders).text
                soup = BeautifulSoup(page_text,'lxml')
                movie_characteristics = soup.find('div', class_= "panel-body content_body")
                if (movie_characteristics == None): 
                    with open('links_to_fix.csv', 'a', newline='') as file:
                        file.write(f"{df.loc[row_index,'Movie_Title']}, {new_link},\n") 
                    continue
        except ValueError: 
            new_link = link+'_'+ str(row['Release_Year']) 
            df.loc[row_index,'Rottentomatoes_URLs'] =  new_link 
            page_text = requests.get(new_link, headers=myheaders).text
            soup = BeautifulSoup(page_text,'lxml')
            movie_characteristics = soup.find('div', class_= "panel-body content_body")
            if (movie_characteristics == None): 
                with open('links_to_fix.csv', 'a', newline='') as file:
                    file.write(f"{df.loc[row_index,'Movie_Title']}, {new_link},\n") 
                continue

fix_urls(df)

Go back to the link generator function for any adjustments

Here below you can see the result.
I made **two links purposefully erronous** to demonstrate that the function works properly: 

In [ ]:
df

### 2.3 Removing some movies 

10 movies were not found on the website and are removed from the dataframe

In [ ]:
#10 rows are removed from the dataframe
df_fixed = pd.read_csv("../Code/Data/df_with_most_links_fixed.csv")
df_fixed = df_fixed.drop('Unnamed: 0', axis=1)
df_links = pd.read_csv("../Code/Data/links_to_fix_original.csv", header=None)
df_links.columns =['Movie_Title', 'Rottentomatoes_URLs']
df = pd.merge(df_links, df_fixed, on=['Movie_Title'], how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)
df = df.drop('Rottentomatoes_URLs_x', axis=1)
df = df.rename(columns={'Rottentomatoes_URLs_y': 'Rottentomatoes_URLs'})
df.to_csv('../Code/Data/df_links_all_ok_new.csv', sep=',',index=True, encoding='utf-8')
df

## 3. Scraping Basic Information About The Movies 

**IMPORTANT NOTE** : As of 18.04.24 the below will no longer work with the URLs we previously defined since Rotten Tomatoes updated the movies' pages. 
As a consequence, we will be using a handful of links gathered from https://web.archive.org/ to demonstrate how the code runs.

In [ ]:
#df= pd.read_csv("../Code/Data/df_links_all_ok.csv")  #<--- this is our data set with all the Oscar movies and their links

In [ ]:
df= pd.read_csv("../Code/Data/df_waybackmachine_links.csv") # <---- this is the csv we will be using to show the code works

In [ ]:
basic_characteristics = ['Rating','Genre', 'Original Language', 'Director', 'Producer','Cast and Crew',
                         'Writer', 'Release Date (Theaters)', 'Release Date (Streaming)', 'Rerelease Date (Theaters)', 'Box Office (Gross USA)',
                         'Runtime', 'Distributor','Production Co', 'Sound Mix', 'Aspect Ratio', 'Synopsis', 'Tomatometer', 'Tomatometer State', 
                         'Audience Score', 'Critics Concensus']
new_vals = [None for item in basic_characteristics]
new_col = dict(zip(basic_characteristics, new_vals))
df = df.assign(**new_col)


In [ ]:

myheaders = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) \ '
              'AppleWebKit/537.36 (KHTML, like Gecko) \ ' 		        
              'Chrome/84.0.4147.89 Safari/537.36'}

headers = {'User-Agent': myheaders}

pd.set_option('mode.chained_assignment', None) 

#Scraping each movie page's information using Beautiful Soup and Requests
def scrape_page_information (df) -> pd.DataFrame:
    """function to scrape the information on the movie's page and place it in a dataframe"""
    
    not_crew_members_list =[]
    
    for row_index, row in df.iterrows():  
        link = row['Rottentomatoes_URLs']
        page_text = requests.get(link, headers=myheaders).text 
        soup = BeautifulSoup(page_text,'lxml')
        
        #We look for the most basic information first (director, producer, runtime, etc.)
        try:
            movie_characteristics = soup.find('div', class_= "panel-body content_body") # in this class we retrieve the basic information
            movie_item_labels = list(movie_characteristics.find_all('b', attrs={"data-qa":"movie-info-item-label"}))
            movie_item_values = movie_characteristics.find_all('span', attrs={"data-qa":"movie-info-item-value"})
        except AttributeError:
            continue
        parsed_data = {}
        for idx,label in enumerate(movie_item_labels):
            label = label.text.rstrip(':')
            lsch = ['\n', '  '] 
            parsed_data[label] = movie_item_values[idx].text
            for ch in lsch:
                parsed_data[label] = parsed_data[label].replace(ch, '')            
            df.loc[row_index, label] = parsed_data[label] 
        
        #We look for the audiencescore, tomatometerscore, tomatometerstate: 
        attrs = soup.find('score-board-deprecated').attrs               
        for attr in ['audiencescore', 'tomatometerscore', 'tomatometerstate']:
            df.loc[row_index, 'Tomatometer'] = attrs['tomatometerscore']
            df.loc[row_index, 'Tomatometer State'] = attrs['tomatometerstate']
            df.loc[row_index, 'Audience Score'] = attrs['audiencescore']

        #We look for the critics consensus 
        consensus = soup.find('span', attrs={'data-qa':'critics-consensus'}) 
        try:
            consensus = consensus.text
            df.loc[row_index, 'Critics Concensus'] = consensus
        except AttributeError:
            df.loc[row_index, 'Critics Concensus'] = 'NaN'
            
        #We look for the synopsis of the movie   
        synopsis = soup.find('p', attrs={'data-qa':'movie-info-synopsis'})
        try:
            synopsis = synopsis.text
            df.loc[row_index, 'Synopsis'] = synopsis
        except AttributeError:
            df.loc[row_index, 'Synopsis'] = 'NaN'

        #We look for all the crew members and their roles/jobs in the movie production 
        crew = soup.css.select('a > p', class_='p--small', attrs={'data-qa':'cast-crew-item-link', 'id':'cast-and-crew', 'href':True})
        role = soup.find_all('p', class_='p--small')
        
        roles_list=[]
        for r in role:
            roles_list.append(r.get_text(strip=True))
               
        crew_list=[]
        for c in crew[1:]:
            crew_list.append(c.get_text(strip=True))

        not_crew_members = crew_list[len(roles_list):]#We append this to 'not_crew_members_list' created for the cleaning section 
        for n in not_crew_members:
            not_crew_members_list.append(n)
        
        crew_list = crew_list[:len(roles_list)] 
       
        tuples = zip(crew_list, roles_list)  
        full_crew = [f'{pair[0]} ({pair[1]})' for pair in tuples] 
        full_crew = ', '.join(map(str, full_crew))
        
        df['Cast and Crew'] = df['Cast and Crew'].astype('object')
        df.at[row_index, 'Cast and Crew'] =full_crew 
        
    return df, not_crew_members_list


df, not_crew_members_list = scrape_page_information(df)


In [ ]:
df

In [ ]:
#To avoid running the function again, we pickle the set of our not_crew_members_list:

not_crew_members_list_set = list(set(not_crew_members_list))

with open('not_crew_pickle_extra.pkl', 'wb') as f:
    pickle.dump(not_crew_members_list_set,f)
f.close()

## 4. Scraping Critics Reviews of the Movies

In [ ]:
df_links = pd.read_csv("../Code/Data/df_links_all_ok.csv")
df_links = df_links.drop('Unnamed: 0', axis=1)
df_links.insert(1, 'Reviews_URLs', True)
for row_index, row in df_links.iterrows():
        link = row['Rottentomatoes_URLs']
        df_links.loc[row_index, 'Reviews_URLs'] = link+'/reviews?intcmp=rt-scorecard_tomatometer-reviews'
links = df_links['Reviews_URLs']
df_links = pd.DataFrame(links)
df_links

In [ ]:
#Scraping all of the critics' comments using Selenium 


chrome_options = webdriver.ChromeOptions()
myheaders = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
chrome_options.add_argument(f"user-agent={myheaders}")
#chrome_options.add_argument('--headless=new')  
headers = {'User-Agent': myheaders}

driver = webdriver.Chrome(options=chrome_options)

def scrape_critics_reviews (df_links, driver) -> pd.DataFrame:
    """function to scrape all the critics reviews by using Selenium and placing them in data frames which we then concatenate into a single
    dataframe"""
    
    data_frames_list=[] #this is where we store the data frames
    
    for row_index, row in df_links.iterrows():
        link = row['Reviews_URLs']
        driver.get(link)
        
        #keep clicking the 'load more' button as long as it exists to get all the comments 
        while True:
            time.sleep(1.5)
            try:
                driver.find_element(By.XPATH, '//*[@id="reviews"]/div[2]/rt-button').click()
            except (NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException):  
                break 

        #scrape comments
        reviews = driver.find_elements(By.CLASS_NAME,'review-row')
        reviews_list = []
    
        for review in reviews:
            try:
                name_critic = review.find_element(By.CSS_SELECTOR,'a.display-name').text 
                name_journal = review.find_element(By.CSS_SELECTOR,'a.publication').text
                comment = review.find_element(By.CSS_SELECTOR,'p.review-text').text
                positive_or_negative = review.find_element(By.CSS_SELECTOR,'score-icon-critic-deprecated[alignment=left]').get_attribute('state')
                date = review.find_element(By.CSS_SELECTOR,'span[data-qa=review-date]').text
            except (NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException):
                continue
            
            reviews_list.append({
                'Date':date, 'Author':name_critic, 'Journal':name_journal, 'Positive_Negative':positive_or_negative, 'Reviews':comment})
            
            df_temp = pd.DataFrame(reviews_list)
            df_temp.insert(1, 'Reviews_URLs', row['Reviews_URLs'])
            
         
        data_frames_list.append(df_temp)

    driver.quit() 
    df_result = pd.concat(data_frames_list, ignore_index=True) 
    return df_result

df_result = scrape_critics_reviews(df_links, driver)
df_result

## 5. Merge the Datasets - Movie Characterisitcs and Critics' Reviews

Finally, we merge the datasets for the movie page information and the dataset for the reviews. The result is the stage 1 dataset.

In [ ]:
df_result = pd.read_csv("../Code/Data/data_all_reviews.csv")
df_moviedata = pd.read_csv("../Code/Data/movie_page_data.csv")
df_moviedata = df_moviedata.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)

In [ ]:
reviews_characteristics = ['Reviews_URLs']
reviews_vals = [None for item in reviews_characteristics]
new_col = dict(zip(reviews_characteristics, reviews_vals))
df_moviedata = df_moviedata.assign(**new_col)
for row_index, row in df_moviedata.iterrows():
        link = row['Rottentomatoes_URLs']
        df_moviedata.loc[row_index, 'Reviews_URLs'] = link+'/reviews?intcmp=rt-scorecard_tomatometer-reviews'

In [ ]:

df_merged = pd.merge(df_moviedata, df_result, on='Reviews_URLs', how='left')
df_merged.to_csv('Vedenikova_Vitalia_studentB_stage1.csv', sep=',',index=True, encoding='utf-8')

## 6. Clean the Resulting Dataset

### 6.1 Encode the CSV file correctly

In [ ]:
df = pd.read_csv('Vedenikova_Vitalia_studentB_stage1.csv', low_memory=False)

In [ ]:
df.to_csv('Vedenikova_Vitalia_studentB_stage3.csv', sep=',',index=False, encoding='utf-8-sig')
df = pd.read_csv('Vedenikova_Vitalia_studentB_stage3.csv', low_memory=False)

### 6.2 Remove Columns containing unnecessary data and give standardized names to columns

In [ ]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1','URL_Movie_Names', 'Reviews_URLs', 'Release_Year',
              'Rerelease Date (Theaters)', 'Release Date (Streaming)', 'Release Date (DVD)','Sound Mix', 'Aspect Ratio', 
              'View the collection'], axis=1)

df = df.rename(columns={'Date':'Review_Date', 'Original Language':'Original_Language', 'Cast and Crew': 'Cast_And_Crew', 
                        'Release Date (Theaters)': 'Release_Date_(Theaters)', 'Box Office (Gross USA)':'Box_Office_(Gross_USA)', 
                        'Production Co':'Production_Co','Tomatometer State':'Tomatometer_State', 
                        'Audience Score':'Audience_Score','Critics Concensus':'Critics_Concensus'})

In [ ]:
#result:
df.columns

### 6.3 Remove the extra spaces (such as \n, \s)

In [ ]:
df['Cast_And_Crew'][2594]

In [ ]:

col_list = ['Synopsis','Cast_And_Crew']
df[col_list] = df[col_list].apply(lambda x : x.str.strip())


df = df.replace(r'\n{1,}\s{2,}', ' ', regex=True)

In [ ]:
#result:
df['Cast_And_Crew'][2594]

### 6.4 Remove Unnecessary Characters and Inaccurate Information for the Crew

In [ ]:

df['Release_Date_(Theaters)'] = df['Release_Date_(Theaters)'].str[0:12]

In [ ]:


#We deserialize our variable:
with open('../Code/Data/not_crew_pickle_extra.pkl', 'rb') as f:
    not_crew_list = pickle.load(f) 
    

In [ ]:

df['Cast_And_Crew'][6915]

In [ ]:

for row_index, row in df.iterrows():
    for element in not_crew_list:
        try: 
            if element in row['Cast_And_Crew']:
                text= row['Cast_And_Crew']
                t = re.sub(rf'{element}.*\)','',text)[0:-2]  
                df.loc[row_index, 'Cast_And_Crew'] = t
        except TypeError:
            continue

In [ ]:
#result
df['Cast_And_Crew'][6915]

### 6.5 Handle Missing Values


In [ ]:

df['Cast_And_Crew'] = df['Cast_And_Crew'].str.replace('()', '(NaN)')


df = df.fillna("NaN")


In [ ]:

review_counts = df["Movie_Title"].value_counts().sort_index()
review_counts = review_counts.to_frame()

subcategory = review_counts[review_counts['count']>1]
subcategory = subcategory.reset_index()
subcategory_list = [movie for movie in subcategory['Movie_Title']]


for row_index, row in df.iterrows():
    if df.at[row_index,'Reviews'] == 'NaN' and row['Movie_Title'] in subcategory_list:
        df.drop(row_index,inplace=True)
        

### 6.6 Standardize Data 

In [ ]:
def standardize_data (df) -> pd.DataFrame:
    '''Standardizes the data in the columns 'Release_Date_(Theaters)','Review_Date', 'Runtime' and 'Box_Office_(Gross_USA)' '''
    
    
    df[['Release_Date_(Theaters)','Review_Date']] = df[['Release_Date_(Theaters)','Review_Date']].apply(lambda x: pd.to_datetime(x, errors='coerce'))

   
    time = df['Runtime'].str.extract('(\d+)h', expand=False).astype(float) * 60 +  df['Runtime'].str.extract('(\d+)m', expand=False).astype(float)
    df['Runtime'] = time

    
    df['Box_Office_(Gross_USA)'] = df['Box_Office_(Gross_USA)'].str.replace('$', '')
    df['Box_Office_(Gross_USA)'] = df['Box_Office_(Gross_USA)'].replace({'K': 'e+03', 'M': 'e+06'}, regex=True).astype(float) 
    
    return df

df = standardize_data(df)

### 6.7 Remove Duplicates
Some reviews appear twice and should be removed. The ones with 'NaN' should be kept.

In [ ]:
df = df.fillna("NaN")
df = pd.concat([df[df['Reviews']=='NaN'], df[df['Reviews']!='NaN'].drop_duplicates(['Reviews'])])

In [ ]:

counts = df["Reviews"].value_counts().sort_index()
counts = counts.to_frame()
reviews_with_duplicates = counts[counts['count']>1]
reviews_with_duplicates

In [ ]:
df.sort_values(['Movie_Title','Review_Date'], inplace=True)

In [ ]:
df.to_csv('Vedenikova_Vitalia_studentB_stage3.csv', sep=',',index=False, encoding='utf-8-sig')